# Data Exploration

In [ ]:
# Import required packages
import pandas as pd

In [21]:
merged_file = "../data/merged_troll_data.json"
troll_2016_no_rt = "../data/troll_data_2016_no_rt.csv"
troll_2016 = "../data/troll_data_2016.csv"
troll_june_nov = "../data/troll_jun_to_nov_v2.csv"

merged_df = pd.read_json(merged_file)

In [20]:
merged_df.iloc[1:10]

,content,followers,following,retweet,account_category,created_at,troll,orig_index
1,Gotta stay high all my life to forget I'm miss...,1148,1000,1,LeftTroll,2016-10-05 13:48:00,True,2162
10,The Issues: Hillary Clinton And Donald Trump O...,1149,1000,1,LeftTroll,2016-10-06 13:58:00,True,2171
100,You'd know how much money I have by how easily...,1151,1001,1,LeftTroll,2016-10-06 20:06:00,True,2262
1000,LIVE on #Periscope: Hillary Workers Intimidate...,1684,1822,1,RightTroll,2016-11-09 01:00:00,True,6294
10000,If THIS is how @HillaryClinton deals with adve...,1942,2515,1,RightTroll,2016-08-24 13:59:00,True,116135
100000,Pretty much https://t.co/HHwVDkOlwS,388,455,1,RightTroll,2016-09-17 03:07:00,True,321552
100001,.@JamesTHarris: âThe border is wide open and...,388,455,1,RightTroll,2016-09-17 03:08:00,True,321553
100002,#CNN says #Hillary team in 2008 never raised #...,388,455,1,RightTroll,2016-09-17 03:09:00,True,321554
100003,I started feeling sorry for politicians this m...,388,455,1,RightTroll,2016-09-17 03:10:00,True,321555
